### Data Preparation

In [47]:
import os
import pandas as pd
from PIL import Image,UnidentifiedImageError
import numpy as np

# Path to the folder containing subfolders for each day
image_folder = './dataset/006_Banqiao'

# Load numerical data
numerical_data = pd.read_csv('./dataset/Banqiao_2022.csv')
#drop the Station column
numerical_data = numerical_data.drop(columns=['Station'])
# Ensure the 'date' column in numerical_data is in datetime format
print(numerical_data.head())
numerical_data['date'] = pd.to_datetime(numerical_data['date'],format="%d-%m-%Y %H:%M",dayfirst=True)

# Function to load images for a specific date
def load_images_for_date(date):
    date_str = date.strftime('%Y%m%d')
    images = []
    for hour in range(24):
        img_name=f"006-{date_str}{hour:02d}00.jpg"
        img_path = os.path.join(image_folder, date_str, img_name)
        try: 
            if os.path.exists(img_path):
                with Image.open(img_path) as img:
                    images.append(img.copy())
            else:
                images.append(None)
        except(OSError,UnidentifiedImageError):
            images.append(None)
    return images

# Create a dictionary to store images by date
image_data = {date: load_images_for_date(date) for date in numerical_data['date'].dt.date.unique()}


               date measurement     0     1     2     3     4     5     6  \
0  01-02-2022 00:00    AMB_TEMP    16    16    16  16.4  16.4  16.7  16.8   
1  01-02-2022 00:00         CH4     2  2.02  2.05  2.04  2.07  2.09  2.08   
2  01-02-2022 00:00          CO  0.24  0.25  0.23   0.2   0.2  0.21  0.21   
3  01-02-2022 00:00        NMHC  0.03  0.04  0.02  0.02  0.03  0.03  0.02   
4  01-02-2022 00:00          NO   0.2   0.6   0.6   0.6   0.5   0.6   0.5   

      7  ...    14    15    16    17    18    19    20    21    22    23  
0  16.9  ...    17  16.6  16.3  16.3  16.3  16.1  16.2  16.5  16.5  16.4  
1  2.11  ...  2.14  2.09  2.04  2.04  2.08  2.09  2.08  2.04  2.01  1.99  
2  0.28  ...  0.51  0.41   0.3  0.34  0.41  0.34  0.35  0.28  0.24  0.22  
3  0.05  ...  0.14   0.1  0.05  0.07  0.12  0.07  0.09  0.04  0.02     0  
4     1  ...   2.7   1.8   1.1   0.9   4.6   0.8   0.8   0.9   0.5   0.5  

[5 rows x 26 columns]


In [48]:
print(numerical_data)

           date measurement     0     1     2     3     4     5     6     7  \
0    2022-02-01    AMB_TEMP    16    16    16  16.4  16.4  16.7  16.8  16.9   
1    2022-02-01         CH4     2  2.02  2.05  2.04  2.07  2.09  2.08  2.11   
2    2022-02-01          CO  0.24  0.25  0.23   0.2   0.2  0.21  0.21  0.28   
3    2022-02-01        NMHC  0.03  0.04  0.02  0.02  0.03  0.03  0.02  0.05   
4    2022-02-01          NO   0.2   0.6   0.6   0.6   0.5   0.6   0.5     1   
...         ...         ...   ...   ...   ...   ...   ...   ...   ...   ...   
1057 2022-03-31         THC  2.11  2.03  1.98  1.99  1.98  2.07     2  2.04   
1058 2022-03-31       WD_HR    58    86    78    73    91    70   112    64   
1059 2022-03-31  WIND_DIREC    62    98    73    97    77    31   103    60   
1060 2022-03-31  WIND_SPEED   1.3   2.1   2.7     2   1.7   0.6   2.2   1.3   
1061 2022-03-31       WS_HR   1.1   1.6   1.9   1.6   1.3   0.8   1.7   1.5   

      ...    14    15    16    17    18    19    20

### Data Preprocessing

In [49]:
import torchvision.transforms as transforms
from torchvision.models import resnet50,ResNet50_Weights
import torch

# Preprocess images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load pretrained ResNet model
resnet = resnet50(weights=ResNet50_Weights.DEFAULT)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove final classification layer

# Extract features from images
def extract_image_features(images):
    features = []
    for img in images:
        if img is not None:
            img = transform(img).unsqueeze(0)  # Add batch dimension
            with torch.no_grad():
                feature = resnet(img).squeeze()  # Remove batch dimension
            features.append(feature.numpy())
        else:
            features.append(np.zeros(2048))  # Handle missing images
    return features

# Extract image features for each date
image_features = {date: extract_image_features(images) for date, images in image_data.items()}
#free up space
del resnet

c:\Users\LENOVO\anaconda3\envs\teep_test1\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\LENOVO\anaconda3\envs\teep_test1\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [50]:
print(image_features)

{datetime.date(2022, 2, 1): [array([0.35567653, 0.45738918, 0.46315253, ..., 0.3067418 , 0.47416142,
       0.36411425], dtype=float32), array([0.3618305 , 0.5117969 , 0.4758631 , ..., 0.3269296 , 0.47762144,
       0.3278998 ], dtype=float32), array([0.35669926, 0.55472535, 0.4340769 , ..., 0.33177543, 0.5071019 ,
       0.30982074], dtype=float32), array([0.36260945, 0.5538877 , 0.45349514, ..., 0.33539355, 0.43378317,
       0.34841663], dtype=float32), array([0.36990723, 0.47247264, 0.45257992, ..., 0.32169518, 0.4513644 ,
       0.3430698 ], dtype=float32), array([0.36674446, 0.51627   , 0.47423798, ..., 0.2987462 , 0.53235936,
       0.33270013], dtype=float32), array([0.3918359 , 0.53798425, 0.44077656, ..., 0.3143041 , 0.42704338,
       0.3355498 ], dtype=float32), array([0.3674608 , 0.5439472 , 0.47748715, ..., 0.3492486 , 0.45591938,
       0.3000529 ], dtype=float32), array([0.36537147, 0.55721533, 0.42972666, ..., 0.2745813 , 0.40856078,
       0.30352128], dtype=float32),

### Combining features

In [51]:
print(numerical_data.shape)

(1062, 26)


In [52]:
# Reshape numerical data to match image data
#drop the Station column
# print(numerical_data.head())
# numerical_data = numerical_data.drop(columns=['Station'])
numerical_data = numerical_data.melt(id_vars=['date', 'measurement'], var_name='hour', value_name='value')
numerical_data['hour'] = numerical_data['hour'].astype(int)

# Create combined features for each date and hour
combined_features = []
targets = []

for idx, row in numerical_data.iterrows():
    date = row['date'].date()
    hour = row['hour']
    numerical_features = np.array(row['value'])
    # print(hour)
    # print(numerical_features)
    # print(image_features[date][23])
    img_features = image_features[date][hour]
    print(img_features)
    print(numerical_features)
    numerical_features=np.array(numerical_features).reshape(1,-1)
    # combined_feature=
    # combined_feature=np.array()
    combined_feature = np.concatenate((numerical_features, img_features),axis=None)
    combined_features.append(combined_feature)
    targets.append(row['value'])  # Assuming the target is the value for that hour

combined_features = np.array(combined_features)
targets = np.array(targets)


[0.35567653 0.45738918 0.46315253 ... 0.3067418  0.47416142 0.36411425]
16
[0.35567653 0.45738918 0.46315253 ... 0.3067418  0.47416142 0.36411425]
2
[0.35567653 0.45738918 0.46315253 ... 0.3067418  0.47416142 0.36411425]
0.24
[0.35567653 0.45738918 0.46315253 ... 0.3067418  0.47416142 0.36411425]
0.03
[0.35567653 0.45738918 0.46315253 ... 0.3067418  0.47416142 0.36411425]
0.2
[0.35567653 0.45738918 0.46315253 ... 0.3067418  0.47416142 0.36411425]
7.1
[0.35567653 0.45738918 0.46315253 ... 0.3067418  0.47416142 0.36411425]
7.4
[0.35567653 0.45738918 0.46315253 ... 0.3067418  0.47416142 0.36411425]
37.9
[0.35567653 0.45738918 0.46315253 ... 0.3067418  0.47416142 0.36411425]
25
[0.35567653 0.45738918 0.46315253 ... 0.3067418  0.47416142 0.36411425]
22
[0.35567653 0.45738918 0.46315253 ... 0.3067418  0.47416142 0.36411425]
1
[0.35567653 0.45738918 0.46315253 ... 0.3067418  0.47416142 0.36411425]
81
[0.35567653 0.45738918 0.46315253 ... 0.3067418  0.47416142 0.36411425]
1.2
[0.35567653 0.457

In [63]:
print(numerical_data)
# targets=np.array(targets).reshape(-1)
print(targets)
targets=targets[:,np.newaxis]
# targets=targets.squeeze()
# targets.reshape(-1,1)
print(targets.shape)

            date measurement  hour value
0     2022-02-01    AMB_TEMP     0    16
1     2022-02-01         CH4     0     2
2     2022-02-01          CO     0  0.24
3     2022-02-01        NMHC     0  0.03
4     2022-02-01          NO     0   0.2
...          ...         ...   ...   ...
25483 2022-03-31         THC    23     2
25484 2022-03-31       WD_HR    23   104
25485 2022-03-31  WIND_DIREC    23   107
25486 2022-03-31  WIND_SPEED    23   1.8
25487 2022-03-31       WS_HR    23   1.8

[25488 rows x 4 columns]
[ 16.     2.     0.24 ... 107.     1.8    1.8 ]
(25488, 1)


In [64]:
print(combined_features.shape)
print(combined_features)
#convert all fields to float
#there are some fields that are not float
def to_float_with_nan(x):
    try:
        return float(x)
    except ValueError:
        return np.nan

# Vectorize the function to apply it to the entire array
vectorized_to_float_with_nan = np.vectorize(to_float_with_nan)

# Replace garbage values with np.nan and convert to float
combined_features = vectorized_to_float_with_nan(combined_features).astype(np.float32)
targets=vectorized_to_float_with_nan(targets).astype(np.float32)

(25488, 2049)
[[ 16.           0.35567653   0.45738918 ...   0.3067418    0.47416142
    0.36411425]
 [  2.           0.35567653   0.45738918 ...   0.3067418    0.47416142
    0.36411425]
 [  0.24         0.35567653   0.45738918 ...   0.3067418    0.47416142
    0.36411425]
 ...
 [107.           0.30877215   0.5673508  ...   0.33922058   0.39269334
    0.32370335]
 [  1.8          0.30877215   0.5673508  ...   0.33922058   0.39269334
    0.32370335]
 [  1.8          0.30877215   0.5673508  ...   0.33922058   0.39269334
    0.32370335]]


In [65]:
print(combined_features)
print(targets.shape)

[[ 16.           0.35567653   0.45738918 ...   0.3067418    0.47416142
    0.36411425]
 [  2.           0.35567653   0.45738918 ...   0.3067418    0.47416142
    0.36411425]
 [  0.24         0.35567653   0.45738918 ...   0.3067418    0.47416142
    0.36411425]
 ...
 [107.           0.30877215   0.5673508  ...   0.33922058   0.39269334
    0.32370335]
 [  1.8          0.30877215   0.5673508  ...   0.33922058   0.39269334
    0.32370335]
 [  1.8          0.30877215   0.5673508  ...   0.33922058   0.39269334
    0.32370335]]
(25488, 1)


### Model Design

In [66]:
print(combined_features.shape)
print(targets.shape)
targets=vectorized_to_float_with_nan(targets).astype(np.float32)

(25488, 2049)
(25488, 1)


In [67]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Define custom dataset
class MultimodalDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

# Split data into train, validation, test sets
train_size = int(0.7 * len(combined_features))
val_size = int(0.1 * len(combined_features))
test_size = len(combined_features) - train_size - val_size

train_dataset = MultimodalDataset(combined_features[:train_size], targets[:train_size])
val_dataset = MultimodalDataset(combined_features[train_size:train_size + val_size], targets[train_size:train_size + val_size])
test_dataset = MultimodalDataset(combined_features[train_size + val_size:], targets[train_size + val_size:])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# Define the multimodal neural network
class MultimodalNet(nn.Module):
    def __init__(self):
        super(MultimodalNet, self).__init__()
        self.fc1_num = nn.Linear(1, 128)  # 1 numerical feature
        self.fc2_num = nn.Linear(128, 64)

        self.fc1_img = nn.Linear(2048, 128)  # ResNet output size
        self.fc2_img = nn.Linear(128, 64)

        self.fc1_combined = nn.Linear(128, 64)
        self.fc2_combined = nn.Linear(64, 1)  # Output size (1 for regression)

    def forward(self, x):
        x_num = x[:, :1]
        x_img = x[:, 1:]

        x_num = torch.relu(self.fc1_num(x_num))
        x_num = torch.relu(self.fc2_num(x_num))

        x_img = torch.relu(self.fc1_img(x_img))
        x_img = torch.relu(self.fc2_img(x_img))

        x_combined = torch.cat((x_num, x_img), dim=1)
        x_combined = torch.relu(self.fc1_combined(x_combined))
        x_combined = self.fc2_combined(x_combined)

        return x_combined

# Instantiate and train the model
model = MultimodalNet()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(20):
    print(f'Epoch {epoch+1}')
    model.train()
    running_loss = 0.0
    i=0
    for inputs, targets in train_loader:
        print(f"Iteration {i}")
        print(inputs.size)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        i+=1
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')


Epoch 1
Iteration 0
<built-in method size of Tensor object at 0x0000023CC7954C50>
Iteration 1
<built-in method size of Tensor object at 0x0000023CC79566F0>
Iteration 2
<built-in method size of Tensor object at 0x0000023CC79565D0>
Iteration 3
<built-in method size of Tensor object at 0x0000023CC79564B0>
Iteration 4
<built-in method size of Tensor object at 0x0000023CC7956390>
Iteration 5
<built-in method size of Tensor object at 0x0000023CC7956270>
Iteration 6
<built-in method size of Tensor object at 0x0000023CC7956150>
Iteration 7
<built-in method size of Tensor object at 0x0000023CC7956030>
Iteration 8
<built-in method size of Tensor object at 0x0000023CC7955F10>
Iteration 9
<built-in method size of Tensor object at 0x0000023CC7955DF0>
Iteration 10
<built-in method size of Tensor object at 0x0000023CC7955CD0>
Iteration 11
<built-in method size of Tensor object at 0x0000023CC7955BB0>
Iteration 12
<built-in method size of Tensor object at 0x0000023CC7955D30>
Iteration 13
<built-in meth

In [68]:
# Evaluation
model.eval()
total_loss = 0.0
with torch.no_grad():
    for inputs, targets in val_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        total_loss += loss.item()
print(f'Validation Loss: {total_loss/len(val_loader)}')
# Deployment
# Implement a system to gather new data, preprocess it, and use the trained model to make predictions


Validation Loss: nan
